# To decide parameters

In [1]:
pip install tabulate


Note: you may need to restart the kernel to use updated packages.


In [2]:
import numpy as np
import pandas as pd
import copy
import os
import random
import math as math
import matplotlib.pylab as plt #Typically matplotlib.pylab is imported as plt (shortened name)
import pdb
from scipy.optimize import minimize
import shutil
import csv
from itertools import permutations 
from itertools import combinations
import csv

from tabulate import tabulate


CRED = '\033[91m'
CEND = '\033[0m'

## Define the payoffs 

The production of collective good depends on X (number of cooperating factions):
$$ P(X) = b* \frac{X^{\kappa}}{X^{\kappa} + X_0^{\kappa}} $$, where $X_0 = x_0 * N$ and $N$ is the total number of factions. 

The division rule is based on the power of all cooperating factions. Let $f_i$ define the power each faction i possesses. 

Let $v_i$ define the share of the collective good goes to a cooperating faction i, where $v_i = \frac{f_i}{\sum_{j \in C} f_j}$. 

$E$ is the endowment shared within the cooperating factions and c is the cost for cooperation. 

Each cooperating faction's payoff is:
$$ \pi_i = R^0_i + v_i *(E + P(X)) - c $$ 

Each defecting faction's payoff is:
$$ \pi_i = R^0_i $$

In the experiment, we want to keep all parameters the same, only vary b for high or low benefit. $N$ = 2 or 4. 

Current issue: if N=2 is a coordination game for low b, the individual payoffs for N=4 are too small for subjects to cooperate.

[Simulations for proper combinations](#Loops )

How different paramters influence the payoffs:

- [$x_0$ and $\kappa$ on the production of the collective good](#On-the-production-of-the-collective-good)

    - $x_0$ determines how many factions need to cooperate to achieve half of the maximal benefit
        - If we hold $x_0$ the same across N=2 and N=4: 
            - production of the colletive good is the same:  $P^{max} = P(X=2|N=2) = P(X=4|N=4)$;
            - but the payoff for each cooperator will be smaller when N=4:
                $$\pi_{i | N=2}  = R^0_i +  \frac{1}{2}*(E + P^{max}) - c  $$
                 $$\pi_{i | N=4}  = R^0_i +  \frac{1}{4}*(E + P^{max}) - c  $$
            - So far, if we decide to have $\pi_{i | N=2} = 90$, $\pi_{i | N=4}$ <=65
            <br> (see the word file "Candidates_full.doc" for more details and examples.)
            <br>     Not diserable
            - If we want to keep $x_0$ (the difficulty to cooperate) the same, may want to change other paramters for N=4
                - A1: Double the benefit of maximal benefit of cooperation.
                - A2: Cut the cost of cooperation into half.
                - A3: Double the endowment for cooperating factions to share.
        - If we hold $X_0$ the same across N=2 and N=4:
            <br> $X_0 = 2 * x_0$ 
            <br> (see the word file "Candidates_full.doc" for more details and examples.)
    - $\kappa$ determines how varied the payoffs for 1,2,3,4 cooperators.

- $b$ determines the maximal benefit of cooperation
    - This is the only parameter should change between high and low benefit
    
Examples and Power Changes for the examples:
- [Keep $x_0$ Constant](#Example1) 
<font color='red'> 'N=4, not much incentive to cooperate, not ideal!'</font>

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 40 | [40, 40, 40] | [47, 47, 47] | [65] |
|   D    | 60 |      60      |      60      |  60  |

- [Keep $x_0$ Constant + Half cost: 1C < 60 ](#Example2.1) 
<font color='red'> 'N=4, maybe hard to achieve cooperation? '</font>

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 53 | [50, 50, 50] | [81, 81, 81] | [75] |
|   D    | 60 |      60      |      60      |  60  |

- [Keep $x_0$ Constant + Half cost: 1C > 60 ](#Example2.2) 
<font color='red'> </font>

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 90 | [50, 50, 50] | [87, 87, 87] | [75] |
|   D    | 60 |      60      |      60      |  60  |


- [Keep $X_0$ Constant :  4C < 60 ](#Example3.1) 
<font color='red'> </font>

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 40 | [90, 90, 90] | [79, 79, 79] | [56] |
|   D    | 60 |      60      |      60      |  -  |


| You \ Everyone Else | 0C |      1C      |      2C      |  3C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 40 | [90, 90, 90] | [79, 79, 79] | [56] |
|   D    | 60 |      60      |      60      |  60  |

- [Keep $X_0$ Constant :  4C > 60 ](#Example3.2) 
<font color='red'>
</font>


| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 40 | [90, 90, 90] | [86, 86, 86] | [74] |
|   D    | 60 |      60      |      60      |  60  |


# Functions

In [3]:
def production(X, n, b, kappa,X0):
    '''X: number of cooperators;
    n: total number of factions in a society
    b: the maximum possible benefit of the collective action
    kappa: steepness parameter, goes to infinity the production becomes a step function
    x0: half effort parameter; x0 = X0/n; The larger X0, the more cooperating factions X are needed to produce the goods'''
#     X0 = x0 * n
    return b* (X**kappa) / (X**kappa + X0 ** kappa)


def payoff(n, b,  r,e, c,  kappa,X0, power=[]):
    '''Return a payoff matrix for X = 0, 1,2,3,...n '''
        
    if power == [] :
        power = np.ones(n) * 1/n
   


    # Only calculate the C payoff:
    # Rows are for 1C, 2C, 3C, 4C...
    Payoffs_C = np.empty(shape=(n,n),dtype='object')
    
    
    for X in range(1,n+1):
        production =     b* (X**kappa) / (X**kappa + X0 ** kappa)        
        group_member = [x for x in range(n)]

        
            
        for i in range(n):
            other_members = copy.deepcopy(group_member)
            other_members.remove(i)
            possible_pays = []
            if X == 1 :
                possible_pays =  np.round(r +  production + e  - c, 1)
            else :    
                for cooperators in combinations(other_members, X-1):
                    total_power = power[i]
                    for j in cooperators:
                        total_power += power[j]
                    if total_power == 0: 
                        v = 1 
                    else :
                        v = power[i] / total_power
                    pay = r + v*( production + e ) - c

                    possible_pays.append(np.round(pay,1))
#                     possible_pays.append(pay)
            Payoffs_C[X-1, i ] = possible_pays
    

    return Payoffs_C

## Optimization to obtain the parameters for the desired payoff matrix

In [49]:
import scipy.optimize as so #typically scipy.optimize is imported as so

def obj(x, option ):
    '''x = [b_l, b_h kappa,x0, r, e, c]'''
    b_l = x[0]
    b_h = x[1]
    kappa = x[2]
    x0 = x[3]
    r = x[4]
    e = x[5]
    c = x[6]
    
    # Option A: make the game as close as to the paper:
    if option == "A":
        x0 = 0.75
        kappa = 2
        
        # Standard:
        # D=40; C_l = 40; CC_l = 60; C_h = 60
    
    
    # When n =2, 
    n=2
    X0 = x0 * n
    D = r
    C_l = r + production(1, n, b_l, kappa,X0)+e-c
    C_h = r + production(1, n, b_h, kappa,X0)+e-c
    CC_l = r+ 1/n *(production(2, n, b_l, kappa,X0)+e)-c
    CC_h = r+ 1/n *(production(2, n, b_h, kappa,X0)+e)-c
    
    
    # Option B: keep everything else constant coordination game
    # When n=4
    n=4
    X0 = x0 * n
    C_n4_l = r + production(1, n, b_l, kappa,X0)+e-c
    C_n4_h = r + production(1, n, b_h, kappa,X0)+e-c

    CC_n4_l = r+ 1/2 *(production(2, n, b_l, kappa,X0)+e)-c
    CCC_n4_l = r+ 1/3 *(production(3, n, b_l, kappa,X0)+e)-c
    CCCC_n4_l = r+ 1/4 *(production(4, n, b_l, kappa,X0)+e)-c
    
    CC_n4_h = r+ 1/2 *(production(2, n, b_h, kappa,X0)+e)-c
    CCC_n4_h = r+ 1/3 *(production(3, n, b_h, kappa,X0)+e)-c
    CCCC_n4_h = r+ 1/4 *(production(4, n, b_h, kappa,X0)+e)-c
    
    
    if option == "A":
    # Standard:
        # D=40; C_l = 40; CC_l = 60; C_h = 60
       # Here option A is looking for N=2, 
    # option A
        goal = (D-40)**2  + (C_l - 40)**2 + (C_n4_h - 20)**2 
#         goal = (D-40)**2 + (C_l -40)**2 + (CC_l-60)**2   + (C_h -40)**2 +(CC_h-140)**2 + (C_n4_h-40)**2 + (CCC_n4_h-60)**2
    # option A'    
#         goal = (D-40)**2 + (C_l -40)**2 +(CC_l-45)**2 + (C_n4_h-40)**2  +(C_h-50)**2+ (CC_h-90)**2 +(CCCC_n4_h-65)**2 + (CCC_n4_h-55)**2 + (CC_n4_h-45)**2
        
#         + + (C_n4_l-40)**2 +(CCC_n4_l-60)**2 +(CCCC_n4_l-90)   +(CCC_n4_h-140)**2
        #         + + (C_n4_l-40)**2 + (C_n4_h-40)**2
    
    elif option == "B":
        # Standard:
        # n =2: D=60, C_l=40, C_h =40, CC_l = 90,  C_n4_l = 40, C_n4_h=40, CCCC_n4_h = 90; CC_h=140,
        
#         goal = (D-60)**2 + (C_l-40)**2 +  (C_h-40)**2 + (CC_l-90)**2 + (C_n4_l-40)**2 + (C_n4_h-40)**2 +(CCCC_n4_h-90)**2 + (CC_h-140)**2 
        goal = (D-60)**2 + (C_l-40)**2 + (C_n4_l-40)**2 + (C_n4_h-40)**2 +(CCCC_n4_h-90)**2 + (CC_h-140)**2  +(C_h-65)**2


    # Option C: N=4, keep b per faction constant
    elif option == "C":
        # When n=4
        n=4
        X0 = x0 * n
        C_n4_l = r + production(1, n, b_l*2, kappa,X0)+e-c
        C_n4_h = r + production(1, n, b_h*2, kappa,X0)+e-c
        
        CC_n4_l = r+ 1/2 *(production(2, n, b_l*2, kappa,X0)+e)-c
        CCC_n4_l = r+ 1/3 *(production(3, n, b_l*2, kappa,X0)+e)-c
        CCCC_n4_l = r+ 1/4 *(production(4, n, b_l*2, kappa,X0)+e)-c


        CC_n4_h = r+ 1/2 *(production(2, n, b_h*2, kappa,X0)+e)-c
        CCC_n4_h = r+ 1/3 *(production(3, n, b_h*2, kappa,X0)+e)-c
        CCCC_n4_h = r+ 1/4 *(production(4, n, b_h*2, kappa,X0)+e)-c
        
        # Standard:
        # D=60, C_l=40, CC_l = 90, C_n4_l = 40, CC_n4_l = 40, CCC_n4_l = 60,   CCCC_n4_l = 90
        # CCC_n4_h =60
        
        goal = (D-60)**2 + (C_l-40)**2 +  (C_h-40)**2 +  (C_n4_l-40)**2 + (CC_n4_l-40)**2 +(CC_l-90)**2  +(CCCC_n4_l-90)**2 +(CCC_n4_l - 60)**2 + (CCC_n4_h-90)**2



    return goal



r = 40
x0 = 0.75
kappa = 12

b = 167

c = 20
e = 20


compare = 1000
sim = 0

while sim <100:
    
    b = random.uniform(0,500)
    kappa = random.uniform(1,15)
    c = random.uniform(0,100)
    e = random.uniform(0,100)
    r = random.uniform(0,100)
    
    
    initial = [b, b*2, kappa,x0, r, e, c]


    bnds = ((0,500),(0,500), (1,15),(0,1),(0,100),(0,500),(0,500))
    sol = so.minimize(obj,initial, args=("A"), bounds=bnds)
    
    if sol.fun < compare  :
        solution= copy.deepcopy(sol)
        sim += 1

        

In [50]:
sol

      fun: 4.7592430763262985e-12
 hess_inv: <7x7 LbfgsInvHessProduct with dtype=float64>
      jac: array([-9.08957123e-07,  3.19977954e-07,  0.00000000e+00,  0.00000000e+00,
        2.77349418e-08,  2.61609072e-07, -2.21609469e-07])
  message: b'CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL'
     nfev: 64
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([191.28276397, 388.56238147,   1.24930994,   0.75      ,
        39.99999988,  75.40026978, 134.25650621])

In [51]:
for ele in solution.x:
    print(round(ele,3))

191.283
388.562
1.249
0.75
40.0
75.4
134.257


In [52]:
### Testing Area: (Chicken game)

b_l = round(solution.x[0],0)
b_h = round(solution.x[1],0)
kappa = round(solution.x[2],3)
x0 = round(solution.x[3],3)
r = round(solution.x[4],3)
e = round(solution.x[5],3) 
c = round(solution.x[6],3) 


################To be used###############################
n = 2
X0 = x0*n

GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
GameOutput_Single(b_h, n, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])

n = 4
X0 = x0*n
GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
GameOutput_Single(b_h, n, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 4, r, e, c, X0, kappa, [])

------------------------------------------------------------
Endowment to share among elites = 75.4
Cost to cooperate = 134.257
Half effort parameter X0  = 1.5
kappa (steepness parameter) = 1.249
Benefit of cooperatoin = 191.0
------------------------------------------------------------
When the power is []; and the faction number is 2
Payoff for faction 1
+-----------------+------+--------+
| Action\Other(s) |  D   |   C    |
+-----------------+------+--------+
|        C        | 53.0 | [-0.3] |
|        D        | 40.0 |  40.0  |
+-----------------+------+--------+
Payoff for faction 2
+-----------------+------+--------+
| Action\Other(s) |  D   |   C    |
+-----------------+------+--------+
|        C        | 53.0 | [-0.3] |
|        D        | 40.0 |  40.0  |
+-----------------+------+--------+
------------------------------------------------------------
Endowment to share among elites = 75.4
Cost to cooperate = 134.257
Half effort parameter X0  = 1.5
kappa (steepness parameter) 

In [785]:
### Testing Area: (Chicken game)
r = 60

x0 = 0.9
kappa = 9


e = 20
c = 20
# b_l = 85
b_l = 167


################To be used###############################
n = 2
X0 = x0*n


GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])




n = 4
X0 = x0*n
GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 4, r, e, c, X0, kappa, [])

------------------------------------------------------------
Endowment to share among elites = 20
Cost to cooperate = 20
Half effort parameter X0  = 1.8
kappa (steepness parameter) = 9
Benefit of cooperatoin = 167
------------------------------------------------------------
When the power is []; and the faction number is 2
Payoff for faction 1
+-----------------+------+---------+
| Action\Other(s) |  D   |    C    |
+-----------------+------+---------+
|        C        | 60.8 | [110.2] |
|        D        |  60  |   60    |
+-----------------+------+---------+
Payoff for faction 2
+-----------------+------+---------+
| Action\Other(s) |  D   |    C    |
+-----------------+------+---------+
|        C        | 60.8 | [110.2] |
|        D        |  60  |   60    |
+-----------------+------+---------+
------------------------------------------------------------
Endowment to share among elites = 20
Cost to cooperate = 20
Half effort parameter X0  = 3.6
kappa (steepness parameter) = 9
Bene

In [388]:
# Enlarging N doesn't necessarily mean better for full cooperation
r = 60


x0 = 0.75
kappa = 2



n=2
X0 = n*x0

# for b in range(3000, )
b_l = 5000
e = 100
c = 20

production(n, b_l,  r,e, c,  kappa,X0)

TypeError: production() takes 5 positional arguments but 7 were given

In [399]:
b =10
production(4,4,b,2,0.75*4)

6.4

## This section is to show how different powers and b will change the game payoffs

In [11]:
def GameOutput_AllOptions(b_l, r, e, c, x0, kappa, power2=[], power4=[]):
    print("------------------------------------------------------------")
    print("Endowment to share among elites = "+str(e))
    print("Cost to cooperate = "+str(c))
    print("Half effort parameter  = "+str(x0))
    print("kappa (steepness parameter) = "+str(kappa))


    print("------------------------------------------------------------")

    print("Benefit of cooperatoin = "+str(b_l))
    print("When the power is "+ str(power2) + "; and the faction number is 2")
    Outcomes = payoff(2, b_l,  r,e, c,  kappa,x0*2, power2)

    i = 0
    print("Payoff for faction "+str(i))
#     risk_dominance = 
    print(tabulate([['C', Outcomes[0,i], Outcomes[1,i] ], 
                    ['D', r,r]], 
                   headers=['Action\Other(s)', 'D','C'],
                  tablefmt='pretty'))

    print("------------------------------------------------------------")
    print("When the power is "+ str(power4) + "; and the faction number is 4")
    print("x0 is consistent (same share)")
    Outcomes = payoff(4, b_l,  r,e, c,  kappa,x0*4, power4)
    i = 0
    print("Payoff for faction "+str(i))
    print(tabulate([['C', Outcomes[0,i], Outcomes[1,i],Outcomes[2,i],Outcomes[3,i] ], 
                    ['D', r,r,r, r]], 
                   headers=['Action\Other(s)', '3D','1C','2C','3C'],
                  tablefmt='pretty'))
    
    print("------------------------------------------------------------")
    print("When the power is "+ str(power4) + "; and the faction number is 4")
    print("A1: x0 is consistent (same share) + double benefit b")
    Outcomes = payoff(4, 2*b_l,  r,e, c,  kappa,x0*4, power4)
    i = 0
    print("Payoff for faction "+str(i))
    print(tabulate([['C', Outcomes[0,i], Outcomes[1,i],Outcomes[2,i],Outcomes[3,i] ], 
                    ['D', r,r,r, r ]], 
                   headers=['Action\Other(s)', '3D','1C','2C','3C'],
                  tablefmt='pretty'))
    
    print("------------------------------------------------------------")
    print("When the power is "+ str(power4) + "; and the faction number is 4")
    print("A2: x0 is consistent (same share) + half cost")
    Outcomes = payoff(4, b_l,  r,e, 0.5*c,  kappa,x0*4, power4)
    i = 0
    print("Payoff for faction "+str(i))
    print(tabulate([['C', Outcomes[0,i], Outcomes[1,i],Outcomes[2,i],Outcomes[3,i] ], 
                    ['D', r,r,r,'-']], 
                   headers=['Action\Other(s)', '3D','1C','2C','3C'],
                  tablefmt='pretty'))
    
    print("------------------------------------------------------------")
    print("When the power is "+ str(power4) + "; and the faction number is 4")
    print("A3: x0 is consistent (same share) + double endowment")
    Outcomes = payoff(4, b_l,  r,2*e, c,  kappa,x0*4, power4)
    i = 0
    print("Payoff for faction "+str(i))
    print(tabulate([['C', Outcomes[0,i], Outcomes[1,i],Outcomes[2,i],Outcomes[3,i] ], 
                    ['D', r,r,r,'-' ]], 
                   headers=['Action\Other(s)', '3D','1C','2C','3C'],
                  tablefmt='pretty'))
    
    print("------------------------------------------------------------")
    print("When the power is "+ str(power4) + "; and the faction number is 4")
    print("X0 is consistent (same difficulty)")
    Outcomes = payoff(4, b_l,  r,e, c,  kappa,x0*2, power4)
    i = 0
    print("Payoff for faction "+str(i))
    print(tabulate([['C', Outcomes[0,i], Outcomes[1,i],Outcomes[2,i],Outcomes[3,i] ], 
                    ['D', r,r,r,'-']], 
                   headers=['Action\Other(s)', '3D','1C','2C','3C'],
                  tablefmt='pretty'))

    
    
def GameOutput_Single(b_l, n, r, e, c, X0, kappa, power=[]):
    print("------------------------------------------------------------")
    print("Endowment to share among elites = "+str(e))
    print("Cost to cooperate = "+str(c))
    print("Half effort parameter X0  = "+str(X0))
    print("kappa (steepness parameter) = "+str(kappa))
    print("Benefit of cooperatoin = "+str(b_l))

    if n == 2:
        print("------------------------------------------------------------")

        
        print("When the power is "+ str(power) + "; and the faction number is 2")
        Outcomes = payoff(2, b_l,  r,e, c,  kappa,X0, power)

        for i in range(n):
            print("Payoff for faction "+str(i+1))
        #     risk_dominance = 
            print(tabulate([['C', Outcomes[0,i], Outcomes[1,i] ], 
                            ['D', r,r]], 
                           headers=['Action\Other(s)', 'D','C'],
                          tablefmt='pretty'))
    else : 
        print("------------------------------------------------------------")
        print("When the power is "+ str(power) + "; and the faction number is 4")
        Outcomes = payoff(4, b_l,  r,e, c,  kappa,X0, power)
        for i in range(n):
            print("Payoff for faction "+str(i+1))
            print(tabulate([['C', Outcomes[0,i], Outcomes[1,i],Outcomes[2,i],Outcomes[3,i] ], 
                            ['D', r,r,r,'-' ]], 
                           headers=['Action\Other(s)', '3D','1C','2C','3C'],
                          tablefmt='pretty'))

    
def GameOutput_SecondRoundResult(b_l, n, r, e, c, X0, kappa, power=[]):
    print("------------------------------------------------------------")
    print("Endowment to share among elites = "+str(e))
    print("Cost to cooperate = "+str(c))
    print("Half effort parameter X0  = "+str(X0))
    print("kappa (steepness parameter) = "+str(kappa))
    print("Benefit of cooperatoin = "+str(b_l))

    if n == 2:
        print("------------------------------------------------------------")
        print("-----------------Round 1-------------------------------------------")
        
        print("When the power is "+ str(power) + "; and the faction number is 2")
        Outcomes = payoff(2, b_l,  r,e, c,  kappa,X0, power=[])

        i = 0
        print("Payoff for each faction ")
        print(tabulate([['C', Outcomes[0,i], Outcomes[1,i] ], 
                        ['D', r,r]], 
                       headers=['Action\Other(s)', 'D','C'],
                      tablefmt='pretty'))
        print("------------------------------------------------------------")
        print("-----------------Round 2-------------------------------------------")
        print("-------N=2, only variation is 1C1D, the first faction chooses C-----------------------------------------------------")
        
        payoff_list = [Outcomes[0,i], r]
        power_1C = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]
        Outcomes_new = payoff(2, b_l,  r,e, c,  kappa,X0, power_1C)
        print("The payoffs are "+ str(payoff_list))
        print("The power is "+ str(power_1C))
        for i in range(n):
            print("Payoff for faction "+str(i+1))
        #     risk_dominance = 
            print(tabulate([['C', Outcomes_new[0,i], Outcomes_new[1,i] ], 
                            ['D', r,r]], 
                           headers=['Action\Other(s)', 'D','C'],
                          tablefmt='pretty'))
#         payoff_list = [40,60,60,60]
#         power_1C = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]

        
        
        
    else : 
        print("------------------------------------------------------------")
        print("When the power is "+ str(power) + "; and the faction number is 4")
        Outcomes = payoff(4, b_l,  r,e, c,  kappa,X0, power)
        i = 0
        print("Payoff for each faction ")
        print(tabulate([['C', Outcomes[0,i], Outcomes[1,i],Outcomes[2,i],Outcomes[3,i] ], 
                        ['D', r,r,r, r]], 
                       headers=['Action\Other(s)', '3D','1C','2C','3C'],
                      tablefmt='pretty'))
            
        print("------------------------------------------------------------")
        print("-----------------Round 2-------------------------------------------")
        print("-------N=4, variations can be: 1C3D, 2C2D, 3C1D-----------------------------------------------------")
        
        variations = 0
        while variations<3:
            if variations==0 :
                print("-------N=4, 1C3D-----------------------------------------------------")
                payoff_list = [Outcomes[0,i], r,r,r]
            elif variations==1:
                print("-------N=4, 2C2D-----------------------------------------------------")
                payoff_list = [Outcomes[1,i][0], Outcomes[1,i][0],r,r]
            elif variations==2 :
                print("-------N=4, 3C1D-----------------------------------------------------")
                payoff_list = [Outcomes[2,i][0], Outcomes[2,i][0],Outcomes[2,i][0],r]
           
        
            power = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]
            Outcomes_new = payoff(4, b_l,  r,e, c,  kappa,X0, power)
            print("The payoffs are "+ str(payoff_list))
            print("The powers are "+ str(power))
            for i in range(n):
                print("Payoff for faction "+str(i+1))
                print(tabulate([['C', Outcomes_new[0,i], Outcomes_new[1,i],Outcomes_new[2,i],Outcomes_new[3,i] ], ['D', r,r,r, r]], 
                           headers=['Action\Other(s)', '3D','1C','2C','3C'],
                          tablefmt='pretty'))

            variations += 1
            

In [49]:
payoff(n, b_l,  r,e, c,  kappa,X0, power=[])[1][1][0]
                                                  

25.1

In [454]:
### Testing Area: (Chicken game)
r = 40

x0 = 0.75
kappa = 2

e = 0
c = 15
b_l = 50

fixed_para = [b_l, kappa,x0, r, e, c ]

################To be used###############################
n = 2
X0 = x0*n


GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])




n = 4
X0 = x0*n
GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 4, r, e, c, X0, kappa, [])                                                  

------------------------------------------------------------
Endowment to share among elites = 0
Cost to cooperate = 15
Half effort parameter X0  = 1.5
kappa (steepness parameter) = 2
Benefit of cooperatoin = 50
------------------------------------------------------------
When the power is []; and the faction number is 2
Payoff for faction 1
+-----------------+------+--------+
| Action\Other(s) |  D   |   C    |
+-----------------+------+--------+
|        C        | 40.4 | [41.0] |
|        D        |  40  |   40   |
+-----------------+------+--------+
Payoff for faction 2
+-----------------+------+--------+
| Action\Other(s) |  D   |   C    |
+-----------------+------+--------+
|        C        | 40.4 | [41.0] |
|        D        |  40  |   40   |
+-----------------+------+--------+
------------------------------------------------------------
Endowment to share among elites = 0
Cost to cooperate = 15
Half effort parameter X0  = 3.0
kappa (steepness parameter) = 2
Benefit of cooperat

## Example1

Keep x0 constant: search “Candidate: x0 is consistent (same share)”
<br> Change the powers to see how the payoff will be different

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 40 | [40, 40, 40] | [47, 47, 47] | [65] |
|   D    | 60 |      60      |      60      |  60  |


In [683]:
r = 60
e = 0
c =20.226

x0 = 0.812
kappa = 12

b_l = 109


# print("Low Benefit")
# GameOutput(b_l, r, e, c, x0, kappa, power2=[], power4=[])

# Start with 1 set of paramters, how the game will evolve for N=2
# print("With N=2, the payoff ") 
# X0 = x0*2
# GameOutput_Single(b_l, 2, r, e, c, X0, kappa, power=[])
# print("If 1C1D: 40 vs 60 ") 
# GameOutput_Single(b_l, 2, r, e, c, X0, kappa, power=[0.4,0.6])
# most likely both will go D, and reset the power to equal, then start over
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, power=[])
# Now for N=4:
# X0 = x0*4
# Try the following combinations
# power_equal = []
# 1C, payoff is [40,60,60,60]
# payoff_list = [40,60,60,60]
# power_1C = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]

################To be used###############################
n = 2
X0 = x0
GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
# # payoff(n, b_l,  r,e, c,  kappa,X0, power=[.43,.57])
# production(2, n, b_l, kappa,X0)
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])

n = 4
X0 = x0*n
GameOutput_Single(b_l, n, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 4, r, e, c, X0, kappa, [])

# print(CRED + 'N=4, no incentive to cooperate, not ideal!' + CEND)

###### Put your new payoff here: 
# payoff_list = [40,40,60,60]
# new_power = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]
# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, new_power)

------------------------------------------------------------
Endowment to share among elites = 0
Cost to cooperate = 20.226
Half effort parameter X0  = 0.812
kappa (steepness parameter) = 12
Benefit of cooperatoin = 109
------------------------------------------------------------
When the power is []; and the faction number is 2
Payoff for faction 1
+-----------------+-------+--------+
| Action\Other(s) |   D   |   C    |
+-----------------+-------+--------+
|        C        | 140.5 | [94.3] |
|        D        |  60   |   60   |
+-----------------+-------+--------+
Payoff for faction 2
+-----------------+-------+--------+
| Action\Other(s) |   D   |   C    |
+-----------------+-------+--------+
|        C        | 140.5 | [94.3] |
|        D        |  60   |   60   |
+-----------------+-------+--------+
------------------------------------------------------------
Endowment to share among elites = 0
Cost to cooperate = 20.226
Half effort parameter X0  = 3.248
kappa (steepness paramete

In [ ]:
production(3, n, b_l*3, kappa,X0)

## Example2.1

Keep $x_0$ constant + Half cost ; N=4, 1C < 60
<br> Change the powers to see how the payoff will be different

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 53 | [50, 50, 50] | [81, 81, 81] | [75] |
|   D    | 60 |      60      |      60      |  60  |


In [ ]:
r = 60
e = 80
c = 180

x0 = 0.65
kappa = 5

b_l = 380
################To be used###############################
# X0 = x0*2
# GameOutput_Single(b_l, 2, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])
# print(CRED + 'N=2, second round, both may D, start over' + CEND)


X0 = x0*4
C = 0.5*c
# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, [])
GameOutput_SecondRoundResult(b_l, 4, r, e,C, X0, kappa, [])
print(CRED + 'N=4, maybe hard to achieve cooperation? ' + CEND)

###### Put your new payoff here: 
# payoff_list = [40,40,60,60]
# new_power = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]
# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, new_power)

## Example2.2

Keep $x_0$ constant + Half cost;  N=4, 1C > 60
<br> Change the powers to see how the payoff will be different

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 90 | [50, 50, 50] | [87, 87, 87] | [75] |
|   D    | 60 |      60      |      60      |  60  |

In [ ]:
r = 60
e = 110
c = 160

x0 = 0.65
kappa = 8

b_l = 280
################To be used###############################
X0 = x0*2
GameOutput_Single(b_l, 2, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])
# print(CRED + 'N=2, second round, both may D, start over' + CEND)

X0 = x0*4
C = 0.5*c
# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, [])
GameOutput_SecondRoundResult(b_l, 4, r, e,C, X0, kappa, [])
print(CRED + 'N=4, if previously 1C, still attractive to C for the 3 defectors; ' + CEND)
print(CRED + 'N=4, if previously 2C, cooperators have higher risk to C; defectors may want to C ' + CEND)
print(CRED + 'N=4, if previously 3C, the only defector does not want to C ' + CEND)
print(CRED + 'N=4, rather nice feature! ' + CEND)

###### Put your new payoff here: 
# payoff_list = [40,40,60,60]
# new_power = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]
# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, new_power)

## Example3.1

Keep $X_0$ constant;  N=4, 4C < 60
<br> Change the powers to see how the payoff will be different


| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 40 | [90, 90, 90] | [79, 79, 79] | [56] |
|   D    | 60 |      60      |      60      |  60  |


In [ ]:
r = 60
e = 50
c = 100

x0 = 0.9
kappa = 4

b_l = 365

################To be used###############################
X0 = x0*2
GameOutput_Single(b_l, 2, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])
# print(CRED + 'N=2, second round, both may D, start over' + CEND)

X0 = x0*2

# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, [])
GameOutput_SecondRoundResult(b_l, 4, r, e,c, X0, kappa, [])
print(CRED + 'N=4, if previously 1C, the cooperator may still want to C; other 3 defectors really want to C; ' + CEND)
print(CRED + 'N=4, if previously 2C, cooperators want to C; defectors will want to D ' + CEND)
print(CRED + 'N=4, if previously 3C, cooperators may want to D; defector really wants to C ' + CEND)
print(CRED + 'N=4, rather nice feature! ' + CEND)

###### Put your new payoff here: 
# payoff_list = [40,40,60,60]
# new_power = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]
# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, new_power)

## Example3.2

Keep $X_0$ constant;  N=4, 4C > 60
<br> Change the powers to see how the payoff will be different

| Action | 1C |      2C      |      3C      |  4C  |
| :- | -: | :-: | :-: | :-: |
|   C    | 40 | [90, 90, 90] | [86, 86, 86] | [74] |
|   D    | 60 |      60      |      60      |  60  |


In [ ]:
r = 60
e = 0
c = 20

x0 = 0.9
kappa = 9

b_l = 70
################To be used###############################
X0 = x0*2
GameOutput_Single(b_l, 2, r, e, c, X0, kappa, [])
# GameOutput_SecondRoundResult(b_l, 2, r, e, c, X0, kappa, [])
# print(CRED + 'N=2, second round, both may D, start over' + CEND)

X0 = x0*2

# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, [])
GameOutput_SecondRoundResult(b_l, 4, r, e,c, X0, kappa, [])
print(CRED + 'N=4, everyone wants to C; ' + CEND)
print(CRED + 'N=4, maybe not as good ' + CEND)

###### Put your new payoff here: 
# payoff_list = [40,40,60,60]
# new_power = [ round(x,2) for x in payoff_list / np.sum(payoff_list) ]
# GameOutput_Single(b_l, 4, r, e, c, X0, kappa, new_power)

## Loops
### Look for proper parameter settings (exhaustive search)

Meet the following standards:
When the power is equally distributed
1. N = 2: payoff of 1C < D and 2C > D, we want D = 60, 1C=40, 2C = 90; 
2. N = 4: payoff for 2C  and higher are above D

### If keep $x_0$, need to change b or c or e

Conditions: when N=4, Pay_1C = 40, Pay_2C = 60, directly comparable to N=2, for now, with some wiggle rooms

In [ ]:
# This is the rough loop
f = open("Parameters_A1.txt","w")
f.write("# This is a rough loop;  \n")
f.write("x0 in [0.5, 0.95] by 0.05;  \n")
f.write("kappa in [1,9] by 1 \n")
f.write("e in [0,300) by 10 \n")
f.write("c in [0,200) by 10 \n")
f.write("b in [0,500) by 10 \n")
r  = 60
candidates = []
# Can be seen later, x0 needs to be at least 0.5 for faction = 2
for x_0 in range(50,100,5):
    x0 = x_0 / 100
    print(x0)
    for kappa in range(1,10):
        for e in range(0,300,10):
            for c in range(0,200,10):
                for b in range(0,500,10):
                    # firstly, consider same x0; can't find a good candidate;
                    # secondly, consider same X0,
                    Pays_2 =  payoff(2, b,  r,e, c,  kappa,x0*2, power=[])
                    Pays_4 =  payoff(4, 2*b,  r,e, c,  kappa,x0*2, power=[])
#                     if abs(Pays_2[0,0] - 40)<=5   and abs(Pays_2[1,0][0] - 90)<=5 and abs(Pays_4[1,0][0] - 40)<=5  and abs(Pays_4[2,0][0] - 90)<=5 :
                    if Pays_2[0,0] == 40  and Pays_2[1,0][0] == 90: 
                        print("fit the requirement:")
                        print([b,  r,e, c,  kappa,x0])
                        candidates.append([Pays_4[1,0],b,  r,e, c,  kappa,x0 ])
f.write("All candidates \n")
f.write(str(candidates))
f.close()

In [ ]:
# This is around one possible combination, for finer loops
f = open("Parameters_2.txt","w")
f.write("# This is for [b,  r,e, c,  kappa,x0] = [140, 60, 0, 20, 9, 0.9] \n")
r  = 60
candidates = []
# Can be seen later, x0 needs to be at least 0.5 for faction = 2
for x_0 in range(85,100):
    x0 = x_0 / 100
    print(x0)
    for kappa_l in range(85,100):
        kappa = kappa_l / 10
        for e in range(0,100,1):
            for c in range(0,50,1):
                for b in range(140,300,1):
                    # firstly, consider same x0; can't find a good candidate;
                    # secondly, consider same X0,
                    Pays_2 =  payoff(2, b,  r,e, c,  kappa,x0*2, power=[])
                    Pays_4 =  payoff(4, b,  r,e, c,  kappa,x0*4, power=[])
#                     if abs(Pays_2[0,0] - 40)<=5   and abs(Pays_2[1,0][0] - 90)<=5 and abs(Pays_4[1,0][0] - 40)<=5  and abs(Pays_4[2,0][0] - 90)<=5 :
                    if Pays_2[0,0] == 40  and Pays_2[1,0][0] == 90: 
                        print("fit the requirement:")
                        print([b,  r,e, c,  kappa,x0])
                        candidates.append([Pays_4[1,0],b,  r,e, c,  kappa,x0 ])
                        if  Pays_4[3,0][0] > 60 :
                            print("3C > 60")
                            X0 = x0*4
                            GameOutput_Single(b_l, 4, r, e, c, X0, kappa, power=[])
                        if Pays_4[2,0][0] > 60 :
                            print("2C > 60")
                            X0 = x0*4
                            GameOutput_Single(b_l, 4, r, e, c, X0, kappa, power=[])
f.write("All candidates \n")
f.write(str(candidates))
f.close()

In [ ]:
for can in candidates:
    b_l = can[1]
    r = can[2]
    e = can[3]
    c = can[4] 
    kappa = can[5]
    x0 = can[6]
#     GameOutput_AllOptions(b_l, r, e, c, x0, kappa, power2=[], power4=[])
    X0 = x0*4
    GameOutput_Single(b_l, 4, r, e, c, X0, kappa, power=[])

In [ ]:
X0 = x0*2
GameOutput_Single(b, 2, r, e, c, X0, kappa, power=[])
GameOutput_Single(2*b, 4, r, e, c, X0, kappa, power=[])

In [ ]:
f = open("income.txt","w") #the first agrument is the file name, 
                           #the second argument is the mode of opening (r -reading only, w - writing only, ... )
f.write("Name,Income\n") #write the first line of the file
for name in income: #loop over all keys of the income dictionary
    f.write(name+","+str(income[name][0])+"\n") #each time write the key and values separated by a comma
f.close() #Check your working directory. You should see the income.txt there.

In [ ]:
# This is to ensure X0 the same (difficulty of cooperation)

r  = 60
candidates = []
# Can be seen later, x0 needs to be at least 0.5 for faction = 2
for x_0 in range(50,100,5):
    x0 = x_0 / 100
    print(x0)
    for kappa in range(1,10):
        for e in range(0,300,10):
            for c in range(0,200,10):
                for b in range(0,500,10):
                    # firstly, consider same x0; can't find a good candidate;
                    # secondly, consider same X0,
                    Pays_2 =  payoff(2, b,  r,e, c,  kappa,x0*2, power=[])
                    Pays_4 =  payoff(4, b,  r,e, c,  kappa,x0*4, power=[])
                    if Pays_2[0,0] == 40   and Pays_2[1,0][0] == 90 and Pays_4[1,0][0] == 60 :
                        print("fit the requirement:")
                        print([b,  r,e, c,  kappa,x0])
                        candidates.append([Pays_4[1,0],b,  r,e, c,  kappa,x0 ])

In [ ]:
for can in candidates:
    b = can[1]
    r = can[2]
    e = can[3]
    c = can[4]
    kappa = can[5]
    x0 = can[6]*2

    power2 = []
    power4 = []
    b_l = b
    b_h = 2*b
    print("---------One Candidate----------------------------------")
    GameOutput(b_l, b_h, r, e, c, x0, kappa, power2=[], power4=[])

In [ ]:
Next step: pick all 90, different combinations, see how they behave. 

In [ ]:
r = 60
e = 17
c = 60

x0 = 0.75 
kappa = 5

b = 200
Pays_2 =  payoff(2, b,  r,e, c,  kappa,x0*2, power=[])
Pays_4 =  payoff(4, b,  r,e, c,  kappa,x0*4, power=[])

In [ ]:
Pays_4

In [ ]:
Pays_2

In [ ]:
candidates

# Show how kappa and $X_0$ influence the production function

# On the production of the collective good
### $$ P(X) = b* \frac{X^{\kappa}}{X^{\kappa} + X_0^{\kappa}} $$, where $X_0 = x_0 * N$ and $N$ is the total number of factions.

### $x_0$ determines how many factions need to cooperate to achieve half of the maximal benefit

#### When N= 2:

When $x_0 <= 0.5$: 1 person can produce over half of the maximal collective product. The game could be a chicken game if $x_0 < 0.5$.

When $x_0 > 0.5$: 2 people will produce more. 1 person produces less than half of the maximum. This is how to make the game a coordination game. The additional production brought by the second cooperator needs to be splitted into half, impossible for the first case to exceed cost of cooperation. 

#### From N=2 to N=4: 

If holding $x_0$ constant: the intuition is that the ratio of cooperating factions in the society to reach at least half of the maximal benefit of cooperation is fixed. 

But then the maximal production will stay the same for N=4, and the cost will be too high for each faction (since we adjusted the paramters with N=2 case, to make the payoff of 1 cooperator below the payoff of defection and 2 cooperators' individual payoff above the defection, the cooperation cost $c > E+P(X=1| N=2 )$ and $c <\frac{1}{2}(E+P(X=2| N=2 ))$ , when N=4 , $P(X=2| N=2) = P(X=4 | N=4)$, but the production of the collective good for N=4 case will be divided into 4 parts, causing the individual payoff to be way lower. Additionally, if we have $x_0 > 0.5$, P(X=1|N=4) < P(X=1|N=2), P(X=2|N=4) < P(X=1|N=2) the payoff for having 1 cooperator in N=2 case gives a payoff of 40, in N=4 case will only give something < 40. Not very comparable.The payoff of having 2 cooperators in N=4 case can't exceed 40 either. Cooerpation will be very difficult.

To make the payoffs more balanced, has to change a parameter, 2b for having double factions; 2E or half of the cost.(Not mentioned in the paper)

If holding $X_0$ constant: the intuition is the difficulty (or the "number" of cooperating factions to reach at least half of the maximal benefit of cooperation) in the soceity is fixed. When there are more factions, the cooperation's difficulty drops. 


### $\kappa$ determines how varied the payoffs for 1,2,3,4 cooperators.

### Holding X0 constant, vary $\kappa$ from 1 to 20

In [ ]:
# Kappa range (N_max) from 0 to 20
N_max = 20

# Fixed parameters here: 
r = 60
e = 50
c = 100

b = 260
x0 = 0.6

payoff_1C2=[]
payoff_2C2=[]

payoff_4_sameX0 =  np.empty(shape=(4,N_max),dtype='object')
payoff_4_samex0 =  np.empty(shape=(4,N_max),dtype='object')    



for x in range(N_max):
    kappa = x 
    payoff_1C2.append(production(1, 2, b, kappa,x0*2))
    payoff_2C2.append(production(2, 2, b, kappa,x0*2))
#     payoff_1C2.append(payoff(2, b,  r,e, c, kappa,x0*2, power2)[0][0])
#     payoff_2C2.append(payoff(2, b,  r,e, c,  kappa,x0*2, power2)[1][0])
    
    for i in range(4):
        payoff_4_sameX0[i,x] = production(i+1, 4, b, kappa,x0*2)
        payoff_4_samex0[i,x] = production(i+1, 4, b, kappa,x0*4)
#         if i == 0 :
#             payoff_4_sameX0[i,x] = payoff(4, b,  r,e, c,  kappa,x0*2, power2)[i][0]
#             payoff_4_samex0[i,x] = payoff(4, b,  r,e, c,  kappa,x0*4, power2)[i][0]
#         else :
#             payoff_4_sameX0[i,x] = payoff(4, b,  r,e, c,  kappa,x0*2, power2)[i][0][0]
#             payoff_4_samex0[i,x] = payoff(4, b,  r,e, c, kappa,x0*4, power2)[i][0][0]
    
    
fig, ax = plt.subplots(3, sharex=True, sharey=True)
ax[0].set_title('n = 2, $x_0$='+str(x0)+', $X_0$='+str(x0*2))
ax[0].plot(np.arange(0,N_max,1), payoff_1C2, ls='-', c="blue", label='1C')
ax[0].plot(np.arange(0,N_max,1), payoff_2C2, ls='-', c="orange", label='2C')
ax[0].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
# fig.suptitle('When b is '+ str(b) +'and $x_0$ is '+str(x0)+', $X_0$ is '+str(x0*2), fontsize=16)

ax[1].set_title('n = 4, same $X_0$')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[0,:], label='1C ')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[1,:], label='2C')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[2,:], label='3C')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[3,:], label='4C')
ax[1].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)

ax[2].set_title('n = 4, same $x_0$')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[0,:], label='1C')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[1,:], label='2C')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[2,:], label='3C')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[3,:], label='4C')
ax[2].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)

for ax in ax.flat:
    ax.set(xlabel='kappa', ylabel='Payoff')
#     ax.set_xticks(np.linspace(0,N_max,2))
#     ax.set_yticks(np.linspace(0,200,21))

fig.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=2, 
                    wspace=1, 
                    hspace=1)


print("Production: smaller kappa is more desired, for larger payoff differences for 1,2,3,4 cooperators")
print('Average pay from the production, when b is '+ str(b))
print("The only changing part is on b, so will cause a change in the production of collective goods")
print("Once we caliberate the parameters for N=2, defection gives 60, 1C gives 40, 2C gives 60:")
print("If we hold x0 the same,  ")

### Holding  $\kappa$  constant, vary $x_0$ from 0.1 to 1

In [ ]:
# x0 range (N_max) from 0 to 10
N_max = 10

# Fixed parameters here: 
r = 60
e = 50
c = 100

b = 260
kappa = 5

payoff_1C2=[]
payoff_2C2=[]

payoff_4_sameX0 =  np.empty(shape=(4,N_max),dtype='object')
payoff_4_samex0 =  np.empty(shape=(4,N_max),dtype='object')    

for x in range(N_max):
    x0 = x / 10
#     payoff_1C2.append(production(1, 2, b, kappa,x0*2))
#     payoff_2C2.append(production(2, 2, b, kappa,x0*2))
    payoff_1C2.append(payoff(2, b,  r,e, c, kappa,x0*2, power2)[0][0])
    payoff_2C2.append(payoff(2, b,  r,e, c,  kappa,x0*2, power2)[1][0])
    
    for i in range(4):
#         payoff_4_sameX0[i,x] = production(i+1, 4, b, kappa,x0*2)
#         payoff_4_samex0[i,x] = production(i+1, 4, b, kappa,x0*4)
        if i == 0 :
            payoff_4_sameX0[i,x] = payoff(4, b,  r,e, c,  kappa,x0*2, power2)[i][0]
            payoff_4_samex0[i,x] = payoff(4, b,  r,e, c,  kappa,x0*4, power2)[i][0]
        else :
            payoff_4_sameX0[i,x] = payoff(4, b,  r,e, c,  kappa,x0*2, power2)[i][0][0]
            payoff_4_samex0[i,x] = payoff(4, b,  r,e, c, kappa,x0*4, power2)[i][0][0]
    
    
fig, ax = plt.subplots(3, sharex=True, sharey=True)
ax[0].set_title('n = 2, $x_0$='+str(x0)+', $X_0$='+str(x0*2))
ax[0].plot(np.arange(0,N_max,1), payoff_1C2, ls='-', c="blue", label='1C')
ax[0].plot(np.arange(0,N_max,1), payoff_2C2, ls='-', c="orange", label='2C')
ax[0].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
# fig.suptitle('When b is '+ str(b) +'and $x_0$ is '+str(x0)+', $X_0$ is '+str(x0*2), fontsize=16)

ax[1].set_title('n = 4, same $X_0$')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[0,:], label='1C ')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[1,:], label='2C')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[2,:], label='3C')
ax[1].plot(np.arange(0,N_max,1), payoff_4_sameX0[3,:], label='4C')
ax[1].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)

ax[2].set_title('n = 4, same $x_0$')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[0,:], label='1C')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[1,:], label='2C')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[2,:], label='3C')
ax[2].plot(np.arange(0,N_max,1), payoff_4_samex0[3,:], label='4C')
ax[2].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)

for ax in ax.flat:
    ax.set(xlabel='$x_0$', ylabel='Payoff')
#     ax.set_xticks(np.linspace(0,N_max,2))
#     ax.set_yticks(np.linspace(0,200,21))

fig.subplots_adjust(left=0.1,
                    bottom=0.1, 
                    right=0.9, 
                    top=2, 
                    wspace=1, 
                    hspace=1)



In [ ]:
# -----------This is where we produce the payoffs for 1,2,3,4 cooperators

payoff_1C2=[]
payoff_2C2=[]

payoff_4_sameX0 =  np.empty(shape=(4,N_max),dtype='object')
payoff_4_samex0 =  np.empty(shape=(4,N_max),dtype='object')    


for x in range(N_max):
#     payoff_1C2.append(production(1, 2, b, x,x0*2))
#     payoff_2C2.append(production(2, 2, b, x,x0*2))
    payoff_1C2.append(payoff(2, b,  r,e, c,  x,x0*2, power2)[0][0])
    payoff_2C2.append(payoff(2, b,  r,e, c,  x,x0*2, power2)[1][0])
    
    for i in range(4):
#         payoff_4_sameX0[i,x] = production(i+1, 4, b, x,x0*2)
#         payoff_4_samex0[i,x] = production(i+1, 4, b, x,x0*4)
        if i == 0 :
            payoff_4_sameX0[i,x] = payoff(4, b,  r,e, c,  x,x0*2, power2)[i][0]
            payoff_4_samex0[i,x] = payoff(4, b,  r,e, c,  x,x0*4, power2)[i][0]
        else :
            payoff_4_sameX0[i,x] = payoff(4, b,  r,e, c,  x,x0*2, power2)[i][0][0]
            payoff_4_samex0[i,x] = payoff(4, b,  r,e, c,  x,x0*4, power2)[i][0][0]
    
    
fig, ax = plt.subplots(2,2, sharex='col', sharey='all')
fig.suptitle("Payoff of Cooperation, b="+str(b))
# ax.set_xticks(np.linspace(0,100,11))
# ax.set_yticks(np.linspace(0,40,21))

ax[0,0].plot(np.arange(0,N_max,1), payoff_1C2, ls='-', c="blue", label='n = 2')
ax[0,0].plot(np.arange(0,N_max,1), payoff_4_sameX0[0,:], label='n = 4, same $X_0$='+str(x0*2))
ax[0,0].plot(np.arange(0,N_max,1), payoff_4_samex0[0,:], label='n = 4, same $x_0$='+str(x0))
ax[0,0].plot(np.arange(0,N_max,1), r*np.ones(N_max), '--',color='red', label='Defection')
ax[0,0].set_title('1 Cooperator')
ax[0,0].set( ylabel='Payoff')


ax[0,1].plot(np.arange(0,N_max,1), payoff_2C2, ls='-', c="orange", label='n = 2')
ax[0,1].plot(np.arange(0,N_max,1), payoff_4_sameX0[1,:], label='n = 4, same $X_0$='+str(x0*2))
ax[0,1].plot(np.arange(0,N_max,1), payoff_4_samex0[1,:], label='n = 4, same $x_0$='+str(x0))
ax[0,1].plot(np.arange(0,N_max,1), r*np.ones(N_max), '--',color='red', label='Defection')
ax[0,1].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
ax[0,1].set_title('2 Cooperators')

ax[1,0].plot(np.arange(0,N_max,1), payoff_4_sameX0[2,:], label='n = 4, same $X_0$='+str(x0*2))
ax[1,0].plot(np.arange(0,N_max,1), payoff_4_samex0[2,:], label='n = 4, same $x_0$='+str(x0))
ax[1,0].plot(np.arange(0,N_max,1), r*np.ones(N_max), '--',color='red', label='Defection')
ax[1,0].set_title('3 Cooperators')
ax[1,0].set(xlabel='kappa',ylabel='Payoff')

ax[1,1].plot(np.arange(0,N_max,1), payoff_4_sameX0[3,:], label='n = 4, same $X_0$='+str(x0*2))
ax[1,1].plot(np.arange(0,N_max,1), payoff_4_samex0[3,:], label='n = 4, same $x_0$='+str(x0))
ax[1,1].plot(np.arange(0,N_max,1), r*np.ones(N_max), '--',color='red', label='Defection')
ax[1,1].legend(bbox_to_anchor=(1.04,0.5), loc="center left", borderaxespad=0)
ax[1,1].set_title('4 Cooperators')
ax[1,1].set(xlabel='kappa')

# for ax in ax.flat:
#     ax.set(xlabel='kappa', ylabel='Payoff')
